In [ ]:
import pandas as pd
import uuid
import random
import json

### Fix on Google Drive

In [ ]:
# df = pd.read_pickle('../data/teaching.pkl')
# sermons = df[df.category=='sermon'].rename(columns={'title':'title_raw'})
# def get_title(s):
#     try:
#         return s[:s.rfind('(')].strip()
#     except:
#         return ''

# def get_passage(s):
#     try:
#         return s[s.rfind('('):].strip().replace('(','').replace(')','')
#     except:
#         return ''
                
# sermons['title'] = sermons.title_raw.apply(lambda x: get_title(x))
# sermons['passage'] = sermons.title_raw.apply(lambda x: get_passage(x))
# sermons = sermons.sort_values('date_str').reset_index(drop=True)
# sermons.to_excel('../data/temp_sermons_for_google_drive.xlsx')

### Prep for Sanity schema

In [ ]:
def new_audio_link(audioLink):
    try:
        return audioLink.split('.org/')[-1].replace('/','___')
    except:
        return ""

In [ ]:
sermons_edited = pd.read_excel('../data/sermons.xlsx')


In [ ]:
sermons_edited = sermons_edited.drop(columns=['Unnamed: 0',])
sermons_edited['mp3File'] = sermons_edited.audioLink.apply(lambda x: new_audio_link(x))
sermons_edited = sermons_edited.drop(columns=['date'])
sermons_edited = sermons_edited.rename(columns={'date_str':'date', 'audioLink':'oldAudioLink'})
sermons_edited = sermons_edited.reset_index().rename(columns={'index':'_id', 'series':'sermonSeries'})
sermons_edited['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(sermons_edited.shape[0])]
sermons_edited['_type'] = 'sermons'
sermons_edited = sermons_edited[['_id', '_type', 'date', 'speaker', 'book', 'passage', 'title', 'sermonSeries', 'mp3File', 'oldAudioLink']]
sermons_edited = sermons_edited.fillna("")

In [ ]:
sermons_edited = sermons_edited[sermons_edited.date <= "2022-10-31"]

sermons_edited['book'] = sermons_edited.book.apply(lambda x: 'Hebrews' if x == 'Hebrew' else x)




In [ ]:
sermons_edited.head(2)

In [ ]:
mp3List = sermons_edited[['_id','mp3File']].rename(columns={'_id':'assetId', 'mp3File':'mp3FileName'}).to_dict('records')
with open('../cms/mp3List.js', 'w') as f:
    f.write(f"export const mp3s = {json.dumps(mp3List)}")

In [ ]:
sermons_edited = pd.read_excel('../data/sermons (2).xlsx')

In [ ]:
sermons_edited['date'] = sermons_edited.date.apply(lambda x: str(x)[:10])
sermons_edited = sermons_edited.drop(columns=['Unnamed: 0','oldAudioLink','newAudioLink'])
sermons_edited = sermons_edited.reset_index().rename(columns={'index':'_id', 'series':'sermonSeries'})
sermons_edited['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(sermons_edited.shape[0])]
sermons_edited['_type'] = 'sermons'
sermons_edited = sermons_edited[['_id', '_type', 'date', 'speaker', 'book', 'passage', 'title', 'sermonSeries', 'audioLink']]
sermons_edited = sermons_edited.fillna("")
sermons_edited = sermons_edited[sermons_edited.speaker.isin(['Todd Wilson','Stephen Hyde'])]
sermons_edited = sermons_edited[sermons_edited.audioLink != ""]

In [ ]:
sermons_edited.to_json("../cms/sermons.ndjson",
           orient="records",
           force_ascii=False,
           lines=True)

### Sunday School for Sanity

In [ ]:
# df = pd.read_pickle('../data/teaching_clean.pkl')
# ss = df[df.category=='sunday school'].rename(columns={'title':'title_raw'})
# def get_title(s):
#     try:
#         return s[:s.rfind('(')].strip()
#     except:
#         return ''

# def get_passage(s):
#     try:
#         return s[s.rfind('('):].strip().replace('(','').replace(')','')
#     except:
#         return ''
                
# ss['title'] = ss.title_raw.apply(lambda x: get_title(x))
# ss['passage'] = ss.title_raw.apply(lambda x: get_passage(x))
# ss = ss.sort_values('date_str').reset_index(drop=True)
# ss.to_excel('../data/temp_ss_for_google_drive.xlsx')

In [ ]:
ss_edited = pd.read_excel('../data/temp_ss_for_google_drive_fixed.xlsx')

In [ ]:
ss_edited.head(1)

In [ ]:
ss_edited = ss_edited.drop(columns=['Unnamed: 0',])
ss_edited['mp3File'] = ss_edited.audioLink.apply(lambda x: new_audio_link(x))
ss_edited = ss_edited.drop(columns=['date'])
ss_edited = ss_edited.rename(columns={'date_str':'date', 'audioLink':'oldAudioLink', 'handoutLink':'oldHandoutLink'})
ss_edited = ss_edited.reset_index().rename(columns={'index':'_id', 'series':'sundaySchoolSeries'})
ss_edited['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(ss_edited.shape[0])]
ss_edited['_type'] = 'sundayschool'
ss_edited = ss_edited[['_id', '_type', 'date', 'speaker', 'book', 'passage', 'title', 'sundaySchoolSeries', 'mp3File', 'oldAudioLink', 'oldHandoutLink']]
ss_edited = ss_edited.fillna("")
ss_edited['date'] = ss_edited.date.apply(lambda x: str(x)[:10])

In [ ]:
# ss_edited = ss_edited[ss_edited.date <= "2022-10-31"]

ss_edited.drop(columns=['mp3File']).to_json("../cms/sundayschool.ndjson",
           orient="records",
           force_ascii=False,
           lines=True)